In [48]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import math
from config import av_api_key as api_key, seeking_alpha_name, seeking_alpha_pw

# Import dependencies for web scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import numpy as np
import os

# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [49]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"STRL"}, 
             {'symbol':"DXPE", 'basis':105.590}, 
             {'symbol':"TSM"}, 
             {'symbol':"WLDN", 'basis':36.080}, 
             {'symbol':"SSRM"}, 
             {'symbol':"LRN", 'basis':107.868}, 
             {'symbol':"UNFI"}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':124.038}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':57.180}, 
             {'symbol':"ITRN"}, 
             {'symbol':"CRDO", 'basis':0.010}, 
             {'symbol':"PYPL", 'basis':68.073}, 
             {'symbol':"ALL", 'basis':193.027}, 
             {'symbol':"LC", 'basis':5.542}, 
             {'symbol':"QTWO", 'basis':79.148}, 
             {'symbol':"CLS", 'basis':0.010}, 
             {'symbol':"CCL", 'basis':0.010}, 
             {'symbol':"AGX"}, 
             {'symbol':"POWL"}, 
             {'symbol':"PPC"}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE", 'basis':116.372}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':65.040}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA", 'basis':92.574}, 
             {'symbol':"TWLO"}, 
             {'symbol':"APP", 'basis':119.318}, 
             {'symbol':"TMUS", 'basis':218.823}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':40.612}, 
             {'symbol':"CAAP", 'basis':21.570}
             ]

# Brokerage
brokerage_funds = [{'symbol':"FRSH", 'basis':13.667}, 
                   {'symbol':"PGY", 'basis':28.919}, 
                   {'symbol':"COMM", 'basis':0.010}, 
                   {'symbol':'FINV', 'basis':8.950}, 
                   {'symbol':"LX", 'basis':6.612}, 
                   {'symbol':'BCS'}, 
                   {'symbol':'PUK'}, 
                   {'symbol':'PSIX'}, 
                   {'symbol':'NGD', 'basis':4.400}, 
                   {'symbol':'GFI'}, 
                   {'symbol':'BKTI'}, 
                   {'symbol':"SSRM", 'basis':12.210}, 
                   {'symbol':"UNFI"}, 
                   {'symbol':"MFC"}, 
                   {'symbol':"EZPW", 'basis':15.773}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"WFC", 'basis':66.893}, 
                   {'symbol':"ITRN"}, 
                   {'symbol':"PYPL", 'basis':68.292}, 
                   {'symbol':"LC"}, 
                   {'symbol':"QTWO", 'basis':76.435}, 
                   {'symbol':"CCL", 'basis':0.010}, 
                   {'symbol':"PPC"}, 
                   {'symbol':"SYF"}, 
                   {'symbol':"ATGE", 'basis':97.175}, 
                   {'symbol':"SKYW", 'basis':74.583}, 
                   {'symbol':"BLBD"}, 
                   {'symbol':"GM"}, 
                   {'symbol':"OKTA", 'basis':99.535}, 
                   {'symbol':"GRBK"}, 
                   {'symbol':"UBER", 'basis':86.860}, 
                   {'symbol':"CAAP"},
                   {'symbol':"QUBT", 'basis':15.469},
                   {'symbol':"RGTI"}, 
                   {'symbol':"FBTC", 'basis':65.965, 'is_etf': True}
                   ]

In [50]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [51]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=.99, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [52]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,FRSH,13.667,12.30,13.94,13.94,14.58,inf,3.738,0.0000,Hold
1,PGY,28.919,28.11,22.91,22.91,25.36,inf,5.990,0.0000,Sell
2,COMM,0.010,15.24,8.69,2.13,9.59,57.70,30.360,0.0000,Sell
3,FINV,8.950,8.96,9.04,19.53,9.54,6.64,1.087,0.0300,Hold
4,LX,6.612,6.80,6.89,21.68,7.38,5.05,0.668,0.0474,Hold
8,NGD,4.400,4.88,4.43,3.09,4.64,24.40,3.507,0.0000,Sell
11,SSRM,12.210,15.56,12.21,21.24,12.92,19.47,0.945,0.0000,Sell
14,EZPW,15.773,15.60,13.48,27.37,14.04,11.85,0.940,0.0000,Sell
15,ARQT,15.996,14.19,13.65,13.65,14.57,inf,12.740,0.0000,Hold
16,WFC,66.893,77.59,76.54,100.89,78.90,13.38,1.516,0.0209,Hold


In [53]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
5,BCS,0.0,20.05,18.10,46.43,18.54,9.23,0.693,0.0043,Sell
6,PUK,0.0,26.00,23.96,28.15,24.48,15.42,1.897,0.0089,Sell
7,PSIX,0.0,96.32,66.80,20.35,75.50,28.24,27.200,0.0000,Sell
9,GFI,0.0,31.12,23.70,16.50,24.74,22.67,5.380,0.0174,Sell
10,BKTI,0.0,40.07,44.17,28.07,48.39,15.36,4.550,0.0000,Hold
12,UNFI,0.0,25.05,24.44,24.44,25.97,inf,0.973,0.0000,Hold
13,MFC,0.0,29.90,30.71,37.64,31.58,13.31,1.626,0.0559,Hold
17,ITRN,0.0,41.30,37.37,29.54,38.94,15.23,4.402,0.0474,Sell
19,LC,0.0,14.86,12.40,16.26,13.18,23.42,1.223,0.0000,Sell
22,PPC,0.0,50.41,45.81,53.39,47.52,9.63,3.175,0.0000,Sell


In [54]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.99, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [55]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
1,DXPE,105.590,115.44,91.84,76.76,97.13,21.03,3.688,0.0000,Sell
3,WLDN,36.080,112.77,70.55,31.09,75.22,67.25,6.710,0.0000,Sell
5,LRN,107.868,150.43,139.43,83.73,146.82,25.12,4.407,0.0000,Sell
8,EAT,124.038,152.09,161.27,37.93,169.05,21.14,26.080,0.0000,Hold
9,EZPW,15.460,15.60,13.48,27.37,14.04,11.85,0.940,0.0000,Sell
10,ARQT,15.932,14.19,13.65,13.65,14.57,inf,12.740,0.0000,Hold
11,WFC,57.180,77.59,76.54,100.89,78.90,13.38,1.516,0.0209,Hold
13,CRDO,0.010,118.57,81.00,6.19,87.09,415.21,30.320,0.0000,Sell
14,PYPL,68.073,67.08,70.59,57.70,73.19,14.49,3.199,0.0000,Hold
15,ALL,193.027,206.50,195.16,246.43,201.49,9.73,2.475,0.0187,Sell


In [56]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,STRL,0.0,289.86,222.23,91.59,234.92,32.90,10.440,0.0000,Sell
2,TSM,0.0,242.09,214.74,92.27,221.56,29.00,8.140,0.0136,Sell
4,SSRM,0.0,15.56,12.21,21.24,12.92,19.47,0.945,0.0000,Sell
6,UNFI,0.0,25.05,24.44,24.44,25.97,inf,0.973,0.0000,Hold
7,MFC,0.0,29.90,30.71,37.64,31.58,13.31,1.626,0.0559,Hold
12,ITRN,0.0,41.30,37.37,29.54,38.94,15.23,4.402,0.0474,Sell
20,AGX,0.0,228.08,206.61,77.28,220.74,33.34,8.960,0.0061,Sell
21,POWL,0.0,264.83,197.34,169.52,210.28,16.92,4.947,0.0046,Sell
22,PPC,0.0,50.41,45.81,53.39,47.52,9.63,3.175,0.0000,Sell
23,SYF,0.0,69.60,63.58,109.49,65.70,8.44,1.642,0.0153,Sell


In [57]:
'''def setup_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome
    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here
    return driver

def scrape_quant_ratings(symbol, driver, days=63):
    url = f'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings'
    driver.get(url)
    time.sleep(5)  # Extra delay for dynamic content
    
    try:
        # Wait for table to load
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
    except Exception as e:
        driver.save_screenshot(f'error_{symbol}.png')
        with open(f'page_source_{symbol}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")
        raise Exception(f"Failed to load page for {symbol}")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find the quant rating history table
    table = soup.find('table')
    
    if not table:
        raise Exception("Table not found")
    
    rows = table.find_all('tr')[1:]  # Skip header
    data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 4:
            date_str = cols[0].text.strip()
            try:
                date = datetime.strptime(date_str, '%m/%d/%Y')
            except ValueError:
                continue
            quant_score_str = cols[3].text.strip()
            try:
                quant_score = float(quant_score_str)
            except ValueError:
                quant_score = np.nan
            data.append({'date': date, 'quant_score': quant_score})
    
    df = pd.DataFrame(data)
    df = df.sort_values('date', ascending=False)
    
    # Filter last 63 days (current date as 02:24 PM MDT, Aug 10, 2025)
    end_date = datetime(2025, 8, 10, 14, 24)  # 2:24 PM MDT
    start_date = end_date - timedelta(days=days)
    df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    return df_filtered

def compute_stats(df):
    scores = df['quant_score'].dropna()
    if len(scores) == 0:
        return {'average': np.nan, 'median': np.nan}
    return {
        'average': scores.mean(),
        'median': scores.median()
    }

# Main function
def main(companies):
    driver = setup_driver()
    try:
        results = {}
        for symbol in companies:
            try:
                df = scrape_quant_ratings(symbol, driver)
                stats = compute_stats(df)
                results[symbol] = stats
            except Exception as e:
                print(f"Error for {symbol}: {e}")
                results[symbol] = {'average': np.nan, 'median': np.nan}
        
        # Output results
        results_df = pd.DataFrame.from_dict(results, orient='index')
        print(results_df)
    finally:
        driver.quit()

# Example usage
companies = ['DXPE', 'AAPL', 'GOOG']  # Replace with your list
main(companies)
'''

'def setup_driver():\n    chrome_options = Options()\n    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")  # Connect to existing Chrome\n    driver = webdriver.Chrome(options=chrome_options)  # No need for user-data-dir here\n    return driver\n\ndef scrape_quant_ratings(symbol, driver, days=63):\n    url = f\'https://seekingalpha.com/symbol/{symbol}/ratings/quant-ratings\'\n    driver.get(url)\n    time.sleep(5)  # Extra delay for dynamic content\n    \n    try:\n        # Wait for table to load\n        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, \'table\')))\n    except Exception as e:\n        driver.save_screenshot(f\'error_{symbol}.png\')\n        with open(f\'page_source_{symbol}.html\', \'w\', encoding=\'utf-8\') as f:\n            f.write(driver.page_source)\n        print(f"Page load failed for {symbol}: {e}. Saved screenshot and page source for debugging.")\n        raise Exception(f"Failed to load page for {symbol

In [58]:
# Machine Learning

In [59]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [60]:
# Feature selection and normalization

In [61]:
# Build model

In [62]:
# Train model

In [63]:
# Evaluate model

In [64]:
# Refine model through hyperparameter tuning